In [2]:
path = '/data2/patho-vit_5_23'
label_csv = None

def exists(val):
    return val is not None

pathid_orig = [f[:-6] for f in os.listdir(os.path.join(path, "store_4")) if not f.startswith(".")]
    # 【:-6】的原因：pyvips切割wsi后，生成的文件自动加上了后缀_files,正好是6个字节
    #pathid_orig = [f[:-6] for f in os.listdir(os.path.join(path, "pseudo_store_3")) if not f.startswith(".")]
    
if exists(label_csv):
    df_label = pd.read_csv(label_csv, index_col = "pathid")
pathid = []
images = []
labels = []
for i, id in enumerate(pathid_orig):
    if len(glob.glob(os.path.join(path, "store_4", str(id) + "_files","0", "*.jpeg"))) != 0:
        pathid.append(id)
        # 若生成的csv中的images为空缺，需检查图片的格式是否正确。例如，需要区分.jpeg和.jpg
        #重要提示：注意最后的文件后缀。images.append(glob.glob(os.path.join(path, "store_3", str(id) + "_files","0", "*.jpeg")))
        images.append(glob.glob(os.path.join(path, "store_4", str(id) + "_files","0", "*.jpeg")))
        if exists(label_csv):
            labels.append(df_label.loc[id]["labels"])
        else:
            labels.append(1e-6)
    else:
        pathid_orig_2 = [f for f in os.listdir(os.path.join(path, "store_4", str(id) + "_files")) if not f.startswith(".")]
        for j, id_2 in enumerate(pathid_orig_2):
            pathid.append(id_2)
            images.append(glob.glob(os.path.join(path, "store_4", str(id) + "_files", str(id_2), "0", "*.jpeg")))
            if exists(label_csv):
                labels.append(df_label.loc[id]["labels"])
            else:
                labels.append(1e-6)
c = {"pathid": pathid, "images": images, "labels": labels}
#df_temp = pd.DataFrame(c)
#df_temp.to_csv("files_label/store_4_temp.csv", index=False, encoding="utf_8_sig")
#torch.save(df.to_dict(orient='list'), "files_label/store_4_temp.db")
#df = pd.read_csv("files_label/store_4_temp.csv", index_col = "pathid")

In [26]:
label_csv = "/data2/patho-vit_5_23_ccsurv/qlyy_surv_label.xlsx"
if exists(label_csv):
    df_label = pd.read_excel(label_csv)

index_train, index_valid = train_test_split(range(df_label['num'].count()), test_size = 0.3)
df_train = df_label[df_label['patientid'].isin(index_train)].reset_index(drop = True)
df_valid = df_label[df_label['patientid'].isin(index_valid)].reset_index(drop = True)

patientid = []
num = []
pathid = []
images = []
labels = []

for i, id in enumerate(df_train.patientid):
    patientid.append(id)

for i, id in enumerate(df_train.num):
    num.append(id)
    
for i, id in enumerate(df_train.pathid):
    pathid.append(id)
    #images.append(df.loc[id]["images"])
    index = c["pathid"].index(id)
    images.append(c["images"][index])
    labels.append(df_train.loc[i].labels)
c2 = {"patientid": patientid, "num": num, "pathid": pathid, "images": images, "labels": labels}
df_train = pd.DataFrame(c2) 

patientid = []
num = []
pathid = []
images = []
labels = []

for i, id in enumerate(df_valid.patientid):
    patientid.append(id)

for i, id in enumerate(df_valid.num):
    num.append(id)
    
for i, id in enumerate(df_valid.pathid):
    pathid.append(id)
    #images.append(df.loc[id]["images"])
    index = c["pathid"].index(id)
    images.append(c["images"][index])
    labels.append(df_valid.loc[i].labels)
c3 = {"patientid": patientid, "num": num, "pathid": pathid, "images": images, "labels": labels}
df_valid = pd.DataFrame(c3) 

df_train.to_csv("/data2/patho-vit_5_23_ccsurv/files_label/store_4_train_ccsurv.csv", index=False, encoding="utf_8_sig")
torch.save(df_train.to_dict(orient='list'), "/data2/patho-vit_5_23_ccsurv/files_label/store_4_train_ccsurv.db")

df_valid.to_csv("/data2/patho-vit_5_23_ccsurv/files_label/store_4_valid_ccsurv.csv", index=False, encoding="utf_8_sig")
torch.save(df_valid.to_dict(orient='list'), "/data2/patho-vit_5_23_ccsurv/files_label/store_4_valid_ccsurv.db")

In [1]:
import os
from sklearn.metrics import balanced_accuracy_score
import gc

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import random
import glob

import torch
import torch.nn as nn
import torch.nn.functional as F

import torchvision.transforms as transforms

from torch.optim.lr_scheduler import StepLR
import torch.backends.cudnn as cudnn

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
path = "/data2/patho-vit_5_23_ccsurv"
# 此处patho_vit后的_5_23为包的版本号
# 若此jupyternotebook运行中kernel挂掉，重启后仅需运行此一代码块，然后跳到需要运行的代码块即可

In [2]:
# 机场2，为每个补丁抽出特征，为整张切片制作特征图像

In [3]:
from patho_vit.vit_luci2 import ViT as ViT
from patho_vit.airport2 import train_features_extractor_gpvit2 as Extractor
from patho_vit.airport2 import valid_features_extractor_gpvit2 as Extractor2
from collections import OrderedDict
path = "/data2/patho-vit_5_23_ccsurv/up"

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from PIL import Image
import cv2
import pandas as pd
import os

In [5]:
def Extractor3(path, train_lib, transform, batch_size, model, device):
    #model = model.to(device)
    #model = nn.DataParallel(model)
    model.eval()

    for i, image in enumerate(train_lib["images"]):
        cols = []
        rows=[]

        for patch in train_lib["images"][i]:
            col, row = patch.split("/")[-1].split(".")[0].split("_")
            cols.append(int(col))
            rows.append(int(row))
        col_last = sorted(np.array(cols))[-1]
        row_last = sorted(np.array(rows))[-1]
        image = torch.zeros((row_last+1), (col_last+1), 768)
    
        patches_batch = []  # 摆渡车
        patches_pos = []  # 补丁位置
        img_features =[]
        for patch in train_lib["images"][i]:
            col, row = patch.split("/")[-1].split(".")[0].split("_")
            patches_pos.append([int(col), int(row)])
    
        for j, patch in enumerate(train_lib["images"][i]):
            torch.cuda.empty_cache() 
            #patch = cv2.imread(patch)
            #patch = cv2.resize(patch, (384, 384))
            #patch = cv2.cvtColor(patch, cv2.COLOR_BGR2RGB)
            patch = Image.open(patch)

            patch = transform(patch).unsqueeze(0) 
            patches_batch.append(patch)
        
            if ((j+1) % batch_size == 0) or ((j+1) == len(train_lib["images"][i])):
                torch.cuda.empty_cache()  
                patches_batch = torch.cat(patches_batch, 0).to(device)
                _, features = model(patches_batch)
                #features = features.softmax(dim = -1)
                features = features.cpu().tolist()
                for k in range(len(features)):
                    img_features.append(features[k])
                patches_batch = []  # 清空摆渡车
    
        for index in range(len(patches_pos)):
            batch_range = torch.arange(patches_pos[index][1], patches_pos[index][1] + 1)[:, None]
            indices = torch.arange(patches_pos[index][0], patches_pos[index][0] + 1)
            image[batch_range, indices] = torch.Tensor(img_features[index]).reshape(1, 1, 768)
    
        #train_lib["images"][i] = image

        torch.save(image, "store_5_96/{}.db".format(train_lib["pathid"][i]))

    images = []
    for i, pathid in enumerate(train_lib["pathid"]):
        images.append(os.path.join(path, "store_5_96", pathid+".db"))
    c = {"pathid": train_lib["pathid"], "images": images, "labels": train_lib["labels"]}
    df = pd.DataFrame(c)
    df.to_csv("files_label/store_5_96_exva.csv", index=False, encoding="utf_8_sig")
    torch.save(c, "files_label/store_5_96_exva.db")
    return "store_5_96_exva.db"

In [6]:
vit = ViT(
    image_size = 384,
    patch_size = 16,
    dim = 768,
    depth = 12,
    heads = 12,
    mlp_dim = 768 * 4,
    num_classes = 2
)
#output, feature = vit(torch.randn(2, 3, 384, 384))
#feature.shape

In [7]:
# 导入定制vit，加载目标编码器的权重

#weights = torch.load("output_jepa/gpvit_weight_4_22_epoch_1.pt")
weights = torch.load("/data2/patho-vit_5_23_ccsurv/up/13/12.4/gpvit_weight_12_4_epoch_18.pt", map_location=torch.device('cpu'))
path = "/data2/patho-vit_5_23_ccsurv/up"
new_dict = OrderedDict()
for k, v in weights.items():
    if "module.target_encoder" in k:
        new_key = k[22:]
        new_dict[new_key] = v

vit = ViT(
    image_size = 384,
    patch_size = 16,
    dim = 768,
    depth = 12,
    heads = 12,
    mlp_dim = 768 * 4,
    num_classes = 1000
)
vit.to(device)
vit.load_state_dict(new_dict, strict = False)
vit = nn.DataParallel(vit)

In [8]:
# 一个文件夹一个文件夹的抽特征，并重构（384，384，768）的图像
transform = transforms.Compose([
    #transforms.Resize([384, 384]),
    transforms.ToTensor(), 
    transforms.Normalize(mean=(0.485, 0.456, 0.406), 
                             std=(0.229, 0.224, 0.225))])

#train_lib = "files/store_5_train_{}.db".format((n+1) * 25)
train_lib = torch.load("/data2/patho-vit_5_23_ccsurv/files_label/store_4_train_ccsurv.db")
valid_lib = torch.load("/data2/patho-vit_5_23_ccsurv/files_label/store_4_valid_ccsurv.db")
exva_lib = torch.load("/data2/patho-vit_5_23_ccsurv/SZL_exva/files/store_4.db")

extractor = Extractor(path = path, train_lib = train_lib, transform = transform, batch_size = 200, model = vit, device = device)
extractor2 = Extractor2(path = path, train_lib = valid_lib, transform = transform, batch_size = 200, model = vit, device = device)
extractor3 = Extractor3(path = path, train_lib = exva_lib, transform = transform, batch_size = 200, model = vit, device = device)

KeyboardInterrupt: 